# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [3]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [4]:
# 1) Seu código aqui
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Munic%C3%ADpio'

In [5]:
def consultar_dados(uf, tipo_interessado):
    base_url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl'
    
    # Parâmetros da consulta
    params = {
        'uf': uf,
        'tipo_interessado': tipo_interessado
    }
    
    # Realiza a requisição HTTP
    response = requests.get(base_url, params=params)
    
    # Verifica se a requisição foi bem-sucedida (código 200)
    if response.status_code == 200:
        # Converte os dados JSON para um DataFrame
        dados_json = response.json()
        df = pd.json_normalize(dados_json['items'])
        return df
    else:
        # Se a requisição não foi bem-sucedida, imprime o código de status
        print(f"Falha na requisição. Código de status: {response.status_code}")
        return None

# Chama a função para consultar os dados
uf_consulta = 'RJ'
tipo_interessado_consulta = 'Município'
df = consultar_dados(uf_consulta, tipo_interessado_consulta)

# Obtém as contagens dos diferentes status
contagem_status = df['status'].value_counts()

# Obtém os três status mais frequentes
tres_status_mais_frequentes = contagem_status.head(3)

# Exibe os resultados
print("Três status mais frequentes e suas frequências:")
print(tres_status_mais_frequentes)


Três status mais frequentes e suas frequências:
Deferido        105
Arquivado        53
Regularizado     26
Name: status, dtype: int64


In [6]:
# Converte a coluna 'data_status' para o tipo datetime
df['data_status'] = pd.to_datetime(df['data_status'], errors='coerce')

# Cria a nova variável 'ano_status'
df['ano_status'] = df['data_status'].dt.year

# Exibe o DataFrame resultante
print(df[['data_status', 'ano_status']])


    data_status  ano_status
0    2014-05-29        2014
1    2010-12-15        2010
2    2023-03-28        2023
3    2022-06-23        2022
4    2023-07-28        2023
..          ...         ...
254  2023-07-31        2023
255  2023-09-25        2023
256  2019-10-17        2019
257  2020-08-21        2020
258  2010-01-12        2010

[259 rows x 2 columns]


C:\Users\mardo\AppData\Local\Temp\ipykernel_9600\4149138174.py:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['data_status'] = pd.to_datetime(df['data_status'], errors='coerce')


####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [6]:
def consultar_dados_uf_tipo(uf, tipo_interessado):
    base_url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl'
    
    # Parâmetros da consulta
    params = {
        'uf': uf,
        'tipo_interessado': tipo_interessado
    }
    
    # Realiza a requisição HTTP
    response = requests.get(base_url, params=params)
    
    # Verifica se a requisição foi bem-sucedida (código 200)
    if response.status_code == 200:
        # Converte os dados JSON para um DataFrame
        dados_json = response.json()
        df = pd.json_normalize(dados_json['items'])
        return df
    else:
        # Se a requisição não foi bem-sucedida, imprime o código de status
        print(f"Falha na requisição. Código de status: {response.status_code}")
        return None

# Exemplo 
uf_consulta = 'RJ'
tipo_interessado_consulta = 'Município'
resultado_df = consultar_dados_uf_tipo(uf_consulta, tipo_interessado_consulta)

# Exibe o DataFrame resultante
print(resultado_df)

     id_pleito tipo_interessado     interessado  cod_ibge  uf  \
0         5275        Município  Rio de Janeiro   3304557  RJ   
1         5618        Município     Nova Iguaçu   3303500  RJ   
2        62993        Município  Paraíba do Sul   3303708  RJ   
3        60039        Município      Petrópolis   3303906  RJ   
4        65517        Município     Nova Iguaçu   3303500  RJ   
..         ...              ...             ...       ...  ..   
254      65888        Município          Paraty   3303807  RJ   
255      66685        Município       Queimados   3304144  RJ   
256      34031        Município        Itatiaia   3302254  RJ   
257      40746        Município     São Gonçalo   3304904  RJ   
258       5734        Município  Rio de Janeiro   3304557  RJ   

                  num_pvl                                             status  \
0                    None  Encaminhado à PGFN com manifestação técnica fa...   
1                    None                                  

In [7]:
# Função para contar as solicitações para o Estado de Minas Gerais com status específico
def contar_solicitacoes_arquivadas(uf, status):
    # Chama a função para consultar os dados para o Estado de Minas Gerais e tipo de interessado 'Estado'
    df = consultar_dados_uf_tipo(uf, 'Estado')

    # Filtra os dados com base no status específico
    df_filtrado = df[df['status'] == status]

    # Obtém o número de solicitações encontradas
    num_solicitacoes = len(df_filtrado)

    # Retorna o resultado
    return num_solicitacoes


uf_consulta = 'MG'
status_consulta = 'Arquivado por decurso de prazo'
num_solicitacoes_arquivadas = contar_solicitacoes_arquivadas(uf_consulta, status_consulta)

# Exibe o resultado
print(f"O número de solicitações para o Estado de Minas Gerais com status '{status_consulta}' é: {num_solicitacoes_arquivadas}")

O número de solicitações para o Estado de Minas Gerais com status 'Arquivado por decurso de prazo' é: 1


In [15]:
def consultar_dados_uf_tipo(uf, tipo_interessado, base_url='https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl'):
    # Parâmetros da consulta
    params = {
        'uf': uf,
        'tipo_interessado': tipo_interessado
    }

    # Realiza a requisição HTTP
    response = requests.get(base_url, params=params)

    # Verifica se a requisição foi bem-sucedida (código 200)
    if response.status_code == 200:
        # Converte os dados JSON para um DataFrame
        dados_json = response.json()
        df = pd.json_normalize(dados_json['items'])

        return df
    else:
        # Se a requisição não foi bem-sucedida, imprime o código de status
        print(f"Falha na requisição. Código de status: {response.status_code}")
        return None

# Função para encontrar o município com mais solicitações deferidas
def municipio_mais_deferido(uf, tipo_interessado):
    # Chama a função para consultar os dados
    df = consultar_dados_uf_tipo(uf, tipo_interessado)

    # Verifica se o DataFrame não está vazio
    if df is None or df.empty:
        print("Nenhum dado encontrado.")
        return None

    # Filtra os dados para incluir apenas as solicitações deferidas
    df_deferidas = df[df['status'].str.lower().str.contains('deferido')]

    # Verifica se há dados após o filtro
    if df_deferidas.empty:
        print("Nenhum município com solicitações deferidas encontrado.")
        return None

    # Encontra o município com mais solicitações deferidas
    municipio_mais_deferido = df_deferidas['interessado'].value_counts().idxmax()

    # Retorna o resultado
    return municipio_mais_deferido


uf_consulta = 'BA'
tipo_interessado_consulta = 'Município'

municipio_resultante = municipio_mais_deferido(uf_consulta, tipo_interessado_consulta)

# Exibe o resultado
if municipio_resultante is not None:
    print(f"O município do estado {uf_consulta} com mais solicitações deferidas é: {municipio_resultante}")

O município do estado BA com mais solicitações deferidas é: Luís Eduardo Magalhães


In [11]:
print(df.columns)

Index(['id_pleito', 'tipo_interessado', 'interessado', 'cod_ibge', 'uf',
       'num_pvl', 'status', 'num_processo', 'data_protocolo', 'tipo_operacao',
       'finalidade', 'tipo_credor', 'credor', 'moeda', 'valor',
       'pvl_assoc_divida', 'pvl_contradado_credor', 'data_status',
       'ano_status'],
      dtype='object')


In [23]:
def consultar_dados_uf_tipo(uf, tipo_interessado, salvar_csv=False, nome_arquivo=None):
    base_url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl'
    
    # Parâmetros da consulta
    params = {
        'uf': uf,
        'tipo_interessado': tipo_interessado
    }
    
    # Realiza a requisição HTTP
    response = requests.get(base_url, params=params)
    
    # Verifica se a requisição foi bem-sucedida (código 200)
    if response.status_code == 200:
        # Converte os dados JSON para um DataFrame
        dados_json = response.json()
        df = pd.json_normalize(dados_json['items'])
        
        # Salva os dados em um arquivo CSV, se solicitado
        if salvar_csv and nome_arquivo:
            df.to_csv(nome_arquivo, index=False)
        
        return df
    else:
        # Se a requisição não foi bem-sucedida, imprime o código de status
        print(f"Falha na requisição. Código de status: {response.status_code}")
        return None


uf_consulta = 'BA'
tipo_interessado_consulta = 'Estado'
nome_arquivo_csv = 'solicitacoes_bahia_estado.csv'

df_bahia_estado = consultar_dados_uf_tipo(uf_consulta, tipo_interessado_consulta, salvar_csv=True, nome_arquivo=nome_arquivo_csv)

# Verifica se o DataFrame foi carregado com sucesso antes de tentar salvar
if df_bahia_estado is not None:
    print(f"Dados salvos com sucesso em '{nome_arquivo_csv}'.")

Dados salvos com sucesso em 'solicitacoes_bahia_estado.csv'.
